In [1]:
import pandas as pd
import re
import tqdm

/Users/devin/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
train_df = pd.read_csv("../data/raw/train.csv")
test_df = pd.read_csv("../data/raw/test.csv")

In [3]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Handle Missing Data

In [4]:
for column in train_df.columns:
    if train_df[column].isna().sum() > 0:
        print(column)

Age
Cabin
Embarked


#### Embarked

In [5]:
train_df['Embarked'].isna().sum()

2

In [6]:
train_df['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [7]:
# fill null value with majority embarked city
train_df['Embarked'] = train_df['Embarked'].fillna('S')
test_df['Embarked'] = train_df['Embarked'].fillna('S')

#### Age

In [8]:
train_df['Age'].isna().sum()

177

In [9]:
train_df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [10]:
train_df['Age'].median()

28.0

In [11]:
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].median())

#### Cabin

In [12]:
train_df['Cabin'].isna().sum()

687

In [13]:
train_df['Cabin'].value_counts()

Cabin
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: count, Length: 147, dtype: int64

In [14]:
imputed_train_df = train_df.drop('Cabin', axis=True)
imputed_test_df = test_df.drop('Cabin', axis=True)

In [15]:
imputed_train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


## Extract Title and Family Name from Name

In [16]:
name_pattern = r'^(?P<family_name>.+),\s(?P<title>Mr\.|Mrs\.|Miss\.|Rev\.|Master\.|Dr\.|Mme\.|Major\.|Sir\.|Mlle\.|Col\.|Capt\.|Don\.|Ms\.|Lady\.|the\sCountess\.|Jonkheer\.)'

In [17]:
def extract_marriage_title(name):
    match = re.search(name_pattern, name)
    if match:
        family_name = match.group('family_name')
        title = match.group('title')
        return pd.Series([family_name, title])
    return pd.Series([None, None])

In [18]:
imputed_train_df[['family_name', 'title']] = imputed_train_df['Name'].apply(extract_marriage_title)
imputed_test_df[['family_name', 'title']] = imputed_test_df['Name'].apply(extract_marriage_title)

In [19]:
imputed_train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family_name,title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Braund,Mr.
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Cumings,Mrs.
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Heikkinen,Miss.
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Futrelle,Mrs.
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Allen,Mr.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,Montvila,Rev.
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,Graham,Miss.
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S,Johnston,Miss.
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,Behr,Mr.


### Extract Ticket title and number from Ticket

In [21]:
ticket_pattern = r'(?P<ticket_prefix>.*\D)?(?P<ticket_number>\d+)'

In [22]:
def extract_ticket(ticket):
    match = re.search(ticket_pattern, ticket)
    if match:
        ticket_prefix = match.group('ticket_prefix')
        ticket_number = match.group('ticket_number')
        return pd.Series([ticket_prefix, ticket_number])
    return pd.Series([None, None])

In [23]:
imputed_train_df[['ticket_prefix', 'ticket_number']] = imputed_train_df['Ticket'].apply(extract_ticket)
imputed_test_df[['ticket_prefix', 'ticket_number']] = imputed_test_df['Ticket'].apply(extract_ticket)

### 

In [24]:
imputed_train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family_name,title,ticket_prefix,ticket_number
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Braund,Mr.,A/5,21171
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Cumings,Mrs.,PC,17599
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Heikkinen,Miss.,STON/O2.,3101282
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Futrelle,Mrs.,None,113803
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Allen,Mr.,None,373450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,Montvila,Rev.,None,211536
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,Graham,Miss.,None,112053
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S,Johnston,Miss.,W./C.,6607
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,Behr,Mr.,None,111369


In [25]:
imputed_train_df['ticket_prefix'].value_counts()

ticket_prefix
PC              60
C.A.            27
STON/O 2.       12
A/5             10
W./C.            9
CA.              8
SOTON/O.Q.       8
A/5.             7
SOTON/OQ         7
STON/O2.         6
CA               6
F.C.C.           5
C                5
SC/PARIS         5
S.O.C.           5
SC/Paris         4
A/4.             3
S.O./P.P.        3
PP               3
A/4              3
A.5.             2
S.C./PARIS       2
SOTON/O2         2
P/PP             2
SC/AH            2
WE/P             2
A./5.            2
SC/AH Basle      1
SO/C             1
S.W./PP          1
F.C.             1
A/S              1
SW/PP            1
SC               1
SCO/W            1
W/C              1
Fa               1
S.O.P.           1
S.C./A.4.        1
A4.              1
S.P.             1
W.E.P.           1
C.A./SOTON       1
Name: count, dtype: int64

## One-hot Encode Categorical Data

In [26]:
encoded_train_df = pd.get_dummies(imputed_train_df, columns=['Sex', 'Embarked', 'ticket_prefix', 'title'])
encoded_test_df = pd.get_dummies(imputed_test_df, columns=['Sex', 'Embarked', 'ticket_prefix', 'title'])

In [27]:
final_train_df = encoded_train_df.drop(['Name', 'Ticket'], axis=1)
final_test_df = encoded_test_df.drop(['Name', 'Ticket'], axis=1)

In [28]:
final_train_df.to_csv('../data/curated/')

IsADirectoryError: [Errno 21] Is a directory: '../'